In [1]:
import pandas as pd
import unicodedata
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
import math

In [ ]:
df = pd.read_excel(
    '/data/total/workspace_luan/ipea/CONCEDIDOS+DADOS+ABERTOS_OUTUBRO+2025.xlsx',
    header=1   
)

In [ ]:
print('Primeiras 5 colunas de dados:\n')
df.head()

## Tratamento dos dados

In [ ]:
df['Competência concessão'].value_counts()

In [ ]:
df.dtypes

In [ ]:
lista_beneficios = df['Espécie.1'].unique()
# benefícios
print(lista_beneficios)
# total benefícios
print(f'\nnúmero de benefícios: {lista_beneficios.__len__()}')

In [ ]:
lista_aposentadorias = [beneficio for beneficio in df['Espécie.1'].dropna().unique() if 'apos' in beneficio.lower()]

In [ ]:
# filtra só aposentadorias
mask_apos = (
    df["Espécie.1"]
    .dropna()
    .str.lower()
    .str.contains("apos")
)

# contagem das espécies
contagem_aposentadorias = df["Espécie.1"][mask_apos].value_counts()

# ajustar nomes das colunas
tabela_contagem_aposentadorias = (
    contagem_aposentadorias
    .reset_index()
    .rename(columns={"Espécie.1": "Espécie"})
)

tabela_contagem_aposentadorias

In [ ]:
# filtrar aposentadorias
df_aposentadorias = df.loc[
    df['Espécie.1'].isin(['Aposentadoria por Idade', 'Aposentadoria por Tempo de Contribuição'])
].copy().reset_index(drop=True)

In [ ]:
print(f'total de casos filtrados: {df_aposentadorias.shape[0]}')

In [ ]:
def calcular_idade(data_nascimento, data_despacho):
    if pd.isna(data_nascimento) or pd.isna(data_despacho):
        return None
    
    idade = data_despacho.year - data_nascimento.year

    # Se ainda não houve aniversário no ano da data de despacho
    if (data_despacho.month, data_despacho.day) < (data_nascimento.month, data_nascimento.day):
        idade -= 1

    return idade

df_aposentadorias["idade"] = [
    calcular_idade(data_nascimento, data_despacho)
    for data_nascimento, data_despacho in zip(
        df_aposentadorias["Dt Nascimento"],
        df_aposentadorias["Dt DDB"]
    )
]

In [ ]:
# idade beneficiário < 50
df_aposentadorias.loc[:, "idade_menor_50"] = (
    df_aposentadorias["idade"] < 50
)


print(
    f"total de casos filtrados com idade menor que 50:\n"
    f"{df_aposentadorias[df_aposentadorias['idade'] < 50].shape[0]}"
)

In [ ]:
df_aposentadorias[['idade_menor_50', 'Classificador PA']].value_counts()

In [ ]:
df_aposentadorias = df_aposentadorias[df_aposentadorias['Classificador PA'] == 'Sem Pensâo Alimentícia']

In [ ]:
df_aposentadorias[df_aposentadorias['idade_menor_50'] == True].shape[0]

In [ ]:
df_aposentadorias = df_aposentadorias.copy()

# flag concessão normal
df_aposentadorias.loc[:, "concessao_normal"] = (
    df_aposentadorias["Despacho.1"] == "Concessao Normal"
)

# value_counts das duas flags juntas
vc_flags = df_aposentadorias[["idade_menor_50", "concessao_normal"]].value_counts()

vc_flags


In [ ]:
df_aposentadorias['Despacho.1'].unique()

In [ ]:
print(f'concessões não normais removidos: {df_aposentadorias[df_aposentadorias["concessao_normal"] == False].shape[0]}')

In [ ]:
df_aposentadorias = df_aposentadorias[df_aposentadorias['Despacho.1'] == 'Concessao Normal']

In [ ]:
df_aposentadorias[df_aposentadorias['idade_menor_50'] == True].shape[0]

In [ ]:
df_aposentadorias.loc[
    df_aposentadorias['idade'] < 50,
    ['Sexo.', 'Espécie.1']
].value_counts()


In [ ]:
df_aposentadorias[df_aposentadorias['Qt SM RMI'] < 1].shape[0]

In [ ]:
df_aposentadorias = df_aposentadorias[df_aposentadorias['Qt SM RMI'] >= 1]

In [ ]:
def classificar_grau(grau):
    if pd.isna(grau):
        return np.nan
    
    s = grau.lower().strip()

    # nível 2 = superior completo
    if "superior completo" in s:
        return 2
    
    # nível 1 = segundo grau completo ou superior incompleto
    elif "colegial completo" in s or "superior incompleto" in s:
        return 1

    # NA
    elif "não informado" in s:
        return np.nan

    # nível 0 = abaixo do segundo grau
    else:
        return 0


# criar coluna numérica
df_aposentadorias["grau_simplificado_indice"] = (
    df_aposentadorias["Grau Instrução"].apply(classificar_grau)
)

# criar coluna com rótulos
df_aposentadorias["grau_simplificado"] = (
    df_aposentadorias["grau_simplificado_indice"]
    .map({
        2: "Superior completo",
        1: "Segundo grau completo",
        0: "Abaixo do segundo grau"
    })
)

df_aposentadorias.loc[
    df_aposentadorias["grau_simplificado_indice"].isna(),
    "grau_simplificado"
] = "Não informado"


In [ ]:
def carregar_tabua(caminho_arquivo):
    """
    Lê a tábua de mortalidade (IBGE), remove linhas não numéricas
    e renomeia as colunas para nomes consistentes.
    """
    
    tabua_crua = pd.read_excel(caminho_arquivo, header=None)

    mascara = tabua_crua[0].apply(
        lambda x: isinstance(x, (int, float)) 
                   and not (isinstance(x, float) and math.isnan(x))
    )
    
    tabua_limpa = tabua_crua.loc[mascara].reset_index(drop=True)

    # renomeia as colunas com nomes consistentes IBGE
    tabua_limpa.columns = [
        "idade",                
        "Q(x,n)_por_mil",       
        "D(x,n)",               
        "l(x)",                 
        "L(x,n)",               
        "T(x)",                 
        "expectativa_sobrevida"      
    ]
    
    return tabua_limpa

In [ ]:
caminho_tabua_homens = "./homens.xlsx"
tabua_homens = carregar_tabua(caminho_tabua_homens)
tabua_homens.head()

In [ ]:
caminho_tabua_mulheres = "./mulheres.xlsx"
tabua_mulheres = carregar_tabua(caminho_tabua_mulheres)
tabua_mulheres.head()

In [ ]:
# garantir que as idades sejam inteiras
tabua_homens["idade"] = tabua_homens["idade"].astype(int)
tabua_mulheres["idade"] = tabua_mulheres["idade"].astype(int)

# mapeamentos idade -> expectativa_sobrevida
mapa_homens = tabua_homens.set_index("idade")["expectativa_sobrevida"]
mapa_mulheres = tabua_mulheres.set_index("idade")["expectativa_sobrevida"]

df_aposentadorias = df_aposentadorias.copy()
df_aposentadorias["expectativa_sobrevida"] = np.nan

mask_h = df_aposentadorias["Sexo."] == "Masculino"
mask_m = df_aposentadorias["Sexo."] == "Feminino"

df_aposentadorias.loc[mask_h, "expectativa_sobrevida"] = (
    df_aposentadorias.loc[mask_h, "idade"].map(mapa_homens)
)

df_aposentadorias.loc[mask_m, "expectativa_sobrevida"] = (
    df_aposentadorias.loc[mask_m, "idade"].map(mapa_mulheres)
)


In [ ]:
df_aposentadorias[["Sexo.", "idade", "expectativa_sobrevida"]].head()

In [ ]:
def norm(s):
    if pd.isna(s):
        return s
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    return "".join(c for c in s if not unicodedata.combining(c))

def criar_regiao(df, uf_col="UF", reg_col="Região"):
    df = df.copy()
    df[reg_col] = df[uf_col].apply(lambda x: mapa_regioes_nomes.get(norm(x), None))
    return df


# mapeia tanto siglas quanto nomes completos (normalizados)
mapa_regioes_nomes = {
    # Norte
    "acre": "Norte",
    "amapa": "Norte",
    "amazonas": "Norte",
    "para": "Norte",
    "rondonia": "Norte",
    "roraima": "Norte",
    "tocantins": "Norte",

    # Nordeste
    "alagoas": "Nordeste",
    "bahia": "Nordeste",
    "ceara": "Nordeste",
    "maranhao": "Nordeste",
    "paraiba": "Nordeste",
    "pernambuco": "Nordeste",
    "piaui": "Nordeste",
    "rio grande do norte": "Nordeste",
    "sergipe": "Nordeste",

    # Centro-Oeste
    "distrito federal": "Centro-Oeste",
    "goias": "Centro-Oeste",
    "mato grosso": "Centro-Oeste",
    "mato grosso do sul": "Centro-Oeste",

    # Sudeste
    "sao paulo": "Sudeste",
    "rio de janeiro": "Sudeste",
    "espirito santo": "Sudeste",
    "minas gerais": "Sudeste",

    # Sul
    "parana": "Sul",
    "rio grande do sul": "Sul",
    "santa catarina": "Sul"
}

# aplicar
df_aposentadorias = criar_regiao(df_aposentadorias, uf_col="UF", reg_col="Região")

In [ ]:
bins = [0, 2, 4, float("inf")]
labels_sal = ["[0,2)", "[2,4)", "4+"]

df_aposentadorias["grupo_renda"] = pd.cut(
    df_aposentadorias["Qt SM RMI"],
    bins=bins,
    labels=labels_sal,
    right=False
)

## Resultados

### Visualizações

In [ ]:
def plot_box_sexo_especie(df):

    grupos = [
        ("Homens\nAposentadoria por idade",
         (df["Sexo."] == "Masculino") & (df["Espécie.1"] == "Aposentadoria por Idade")),

        ("Homens\nAposentadoria por tempo de contribuição",
         (df["Sexo."] == "Masculino") & (df["Espécie.1"] == "Aposentadoria por Tempo de Contribuição")),

        ("Mulheres\nAposentadoria por idade",
         (df["Sexo."] == "Feminino") & (df["Espécie.1"] == "Aposentadoria por Idade")),

        ("Mulheres\nAposentadoria por tempo de contribuição",
         (df["Sexo."] == "Feminino") & (df["Espécie.1"] == "Aposentadoria por Tempo de Contribuição")),
    ]

    labels = [g[0] for g in grupos]
    data = [df[g[1]]["idade"].dropna() for g in grupos]

    stats = []
    for serie in data:
        N = len(serie)
        Q1 = serie.quantile(0.25)
        Q2 = serie.quantile(0.50)
        Q3 = serie.quantile(0.75)
        stats.append((N, Q1, Q2, Q3))

    plt.figure(figsize=(12, 6))
    box = plt.boxplot(data, labels=labels, patch_artist=True)

    for patch in box["boxes"]:
        patch.set(facecolor="#A0C4FF", alpha=0.8)

    plt.ylabel("Idade na data de despacho do benefíco (anos)", fontsize=14)
    plt.grid(axis="y", linestyle="--", alpha=0.4)

    y_max = max(s.max() for s in data if len(s) > 0)
    y_fixed = y_max + 3

    for i, (N, Q1, Q2, Q3) in enumerate(stats, start=1):
        texto = (
            f"N={N}\n"
            f"Q1={Q1:.1f}\n"
            f"Q2 (mediana)={Q2:.1f}\n"
            f"Q3={Q3:.1f}"
        )
        plt.text(
            i, y_fixed, texto,
            ha="center", va="bottom", fontsize=10, color="black"
        )

    plt.ylim(top=117.5)
    plt.tight_layout()
    plt.show()

plot_box_sexo_especie(df_aposentadorias)

In [ ]:

def plot_boxplot_idades_por_salario(df, labels, idade_col="idade"):

    data = [df[df["grupo_renda"] == g][idade_col].dropna() for g in labels]

    stats = []
    for serie in data:
        N = len(serie)
        Q1 = serie.quantile(0.25)
        Q2 = serie.quantile(0.50)
        Q3 = serie.quantile(0.75)
        stats.append((N, Q1, Q2, Q3))

    plt.figure(figsize=(13, 7))
    box = plt.boxplot(data, labels=labels, patch_artist=True)

    for patch in box["boxes"]:
        patch.set(facecolor="#A0C4FF", alpha=0.8)

    plt.xlabel("Quantidade de salários mínimos - renda mensal inicial", fontsize=14)
    plt.ylabel("Idade na data de despacho do benefíco (anos)", fontsize=14)
    plt.grid(axis="y", linestyle="--", alpha=0.4)

    top_whiskers = [max(serie) if len(serie) else 0 for serie in data]
    y_fixed = max(top_whiskers) + 3  

    for i, (N, Q1, Q2, Q3) in enumerate(stats, start=1):
        texto = (
            f"N = {N}\n"
            f"Q1 = {Q1:.1f}\n"
            f"Mediana = {Q2:.1f}\n"
            f"Q3 = {Q3:.1f}"
        )

        plt.text(
            i, y_fixed,
            texto,
            ha="center", va="bottom",
            fontsize=10
        )

    plt.ylim(top = 115.5)

    plt.tight_layout()
    plt.show()

labels_sal = ["[0,2)", "[2,4)", "4+"]

plot_boxplot_idades_por_salario(df_aposentadorias, labels_sal)


In [ ]:
def plot_box_expectativa(df):
    grupos = [
        ("Homens\nAposentadoria por Idade",
         (df["Sexo."]=="Masculino") & (df["Espécie.1"]=="Aposentadoria por Idade")),

        ("Homens\nAposentadoria por Tempo de Contribuição",
         (df["Sexo."]=="Masculino") & (df["Espécie.1"]=="Aposentadoria por Tempo de Contribuição")),

        ("Mulheres\nAposentadoria por Idade",
         (df["Sexo."]=="Feminino") & (df["Espécie.1"]=="Aposentadoria por Idade")),

        ("Mulheres\nAposentadoria por Tempo de Contribuição",
         (df["Sexo."]=="Feminino") & (df["Espécie.1"]=="Aposentadoria por Tempo de Contribuição")),
    ]

    data = [df[filtro]["expectativa_sobrevida"].dropna() for _, filtro in grupos]
    labels = [nome for nome, _ in grupos]

    stats = []
    for serie in data:
        N = len(serie)
        Q1 = serie.quantile(0.25)
        Q2 = serie.quantile(0.50)
        Q3 = serie.quantile(0.75)
        stats.append((N, Q1, Q2, Q3))

    plt.figure(figsize=(12, 6))
    box = plt.boxplot(data, labels=labels, patch_artist=True)

    for patch in box["boxes"]:
        patch.set(facecolor="#A0C4FF", alpha=0.8)

    plt.ylabel("Expectativa de sobrevida (anos)")
    plt.grid(axis="y", linestyle="--", alpha=0.4)

    y_max = max(s.max() for s in data if len(s) > 0)
    y_fixed = 43

    for i, (N, Q1, Q2, Q3) in enumerate(stats, start=1):
        texto = (
            f"N={N}\n"
            f"Q1={Q1:.1f}\n"
            f"Mediana={Q2:.1f}\n"
            f"Q3={Q3:.1f}"
        )
        plt.text(
            i, y_fixed,
            texto,
            ha="center", va="bottom",
            fontsize=10, color="black"
        )

    plt.ylim(top=49.9)

    plt.tight_layout()
    plt.savefig('./figuras/box1_expectativa_sexo_espécie.png')
    plt.show()

plot_box_expectativa(df_aposentadorias)

In [ ]:
def plot_boxplot_expectativa_por_salario(df, labels, col="expectativa_sobrevida"):

    df = df.copy()
    df[col] = pd.to_numeric(df[col], errors="coerce")

    data = [df[df["grupo_renda"] == g][col].dropna() for g in labels]

    stats = []
    for serie in data:
        N = len(serie)
        Q1 = serie.quantile(0.25)
        Q2 = serie.quantile(0.50)
        Q3 = serie.quantile(0.75)
        stats.append((N, Q1, Q2, Q3))

    plt.figure(figsize=(13, 7))
    box = plt.boxplot(data, labels=labels, patch_artist=True)

    for patch in box["boxes"]:
        patch.set(facecolor="#A0C4FF", alpha=0.8)

    plt.xlabel("Quantidade de salários mínimos - renda mensal inicial", fontsize=14)
    plt.ylabel("Expectativa de Vida (anos)", fontsize=14)
    plt.grid(axis="y", linestyle="--", alpha=0.4)

    top_whiskers = [serie.max() if len(serie) else 0 for serie in data]
    y_fixed = max(top_whiskers) + 3

    for i, (N, Q1, Q2, Q3) in enumerate(stats, start=1):
        texto = (
            f"N = {N}\n"
            f"Q1 = {Q1:.1f}\n"
            f"Mediana = {Q2:.1f}\n"
            f"Q3 = {Q3:.1f}"
        )
        plt.text(
            i, y_fixed,
            texto,
            ha="center", va="bottom",
            fontsize=10
        )

    plt.ylim(top = 49.9)

    plt.tight_layout()
    plt.savefig('./figuras/box2_expectativa_grupo_SM.png')
    plt.show()

labels_sal = ["[0,2)", "[2,4)", "4+"]

plot_boxplot_expectativa_por_salario(df_aposentadorias, labels_sal)


In [ ]:
def plot_cdf_expectativa_por_salario_por_sexo(df, sexo, col="expectativa_sobrevida", corte_eixo_x=None):
    """
    Plota curvas de CDF da expectativa de vida separadas por sexo.
    Agora marca corte_eixo_x com uma linha vertical, setas pontilhadas coloridas
    e texto 'x = valor' no rodapé da linha vertical.
    """
    df = df.copy()
    df = df[df["Sexo."] == sexo].copy()
    df[col] = pd.to_numeric(df[col], errors="coerce")

    labels_sal = ["[0,2)", "[2,4)", "4+"]

    plt.figure(figsize=(10, 7))

    cdfs_info = {}
    color_map = {}

    for grupo in labels_sal:
        sub = df[df["grupo_renda"] == grupo].copy()
        if sub.empty:
            continue

        sub["exp_round"] = sub[col].round(1)

        counts = sub["exp_round"].value_counts().sort_index()
        perc = counts / counts.sum() * 100
        cdf = perc.cumsum()

        line, = plt.plot(
            cdf.index,
            cdf.values,
            marker="o",
            linewidth=1.5,
            label=grupo
        )

        cdfs_info[grupo] = (cdf.index.values, cdf.values)
        color_map[grupo] = line.get_color()

    # vertical line + arrows + text
    if corte_eixo_x is not None:
        plt.axvline(corte_eixo_x, linestyle="--", color="black", alpha=0.7)

        # --- adicionar texto "x = valor" no canto inferior da linha ---
        ymin, ymax = plt.ylim()
        plt.text(
            corte_eixo_x + (plt.xlim()[1] - plt.xlim()[0])/12, ymin + (ymax - ymin)*0.02,
            f"expectativa = {corte_eixo_x}",
            ha="center",
            va="bottom",
            fontsize=11,
            color="black",
            bbox=dict(facecolor="white", alpha=0.6, edgecolor="none")
        )

        for grupo, (xs, ys) in cdfs_info.items():
            y_interp = np.interp(corte_eixo_x, xs, ys)
            color = color_map[grupo]

            # seta pontilhada (linha tracejada)
            plt.annotate(
                "",
                xy=(corte_eixo_x + 3, y_interp),
                xytext=(corte_eixo_x, y_interp),
                arrowprops=dict(
                    arrowstyle="->",
                    color=color,
                    lw=2,
                    linestyle="dotted"
                ),
                va="center"
            )

            plt.text(
                corte_eixo_x + 3.1,
                y_interp,
                f"{y_interp:.1f}%",
                fontsize=11,
                va="center",
                ha="left",
                color=color
            )

    plt.xlabel("Expectativa de sobrevida (anos)", fontsize=15)
    plt.ylabel("Percentual acumulado (%)", fontsize=15)
    #plt.title(f"Distribuição Acumulada da Expectativa de Vida – {sexo}", fontsize=16)
    plt.grid(linestyle="--", alpha=0.4)

    leg = plt.legend(title="Quantidade de salários mínimos \nrenda mensal inicial")
    leg.get_title().set_ha("center")


    plt.tight_layout()
    plt.savefig(f'figuras/dist_acum_expectativa_sobrevida_{sexo}_{corte_eixo_x}.png')
    plt.show()


In [ ]:
plot_cdf_expectativa_por_salario_por_sexo(df_aposentadorias, "Masculino",corte_eixo_x = 20)
plot_cdf_expectativa_por_salario_por_sexo(df_aposentadorias, "Feminino",corte_eixo_x = 25)


In [ ]:
def plot_boxplot_idades_por_instrucao(df, instr_labels, idade_col="expectativa_sobrevida"):

    data = [df[df["grau_simplificado"] == g][idade_col].dropna() 
            for g in instr_labels]

    stats = []
    for serie in data:
        N = len(serie)
        Q1 = serie.quantile(0.25)
        Q2 = serie.quantile(0.50)
        Q3 = serie.quantile(0.75)
        stats.append((N, Q1, Q2, Q3))

    plt.figure(figsize=(13, 7))
    box = plt.boxplot(data, labels=instr_labels, patch_artist=True)

    for patch in box["boxes"]:
        patch.set(facecolor="#A0C4FF", alpha=0.8)

    plt.xlabel("Grau de Instrução (agrupado)", fontsize=14)
    plt.ylabel("Expectativa de sobrevida (anos)", fontsize=14)
    plt.grid(axis="y", linestyle="--", alpha=0.4)

    top_whiskers = [serie.max() if len(serie) else 0 for serie in data]
    y_fixed = max(top_whiskers) + 3

    for i, (N, Q1, Q2, Q3) in enumerate(stats, start=1):
        texto = (
            f"N = {N}\n"
            f"Q1 = {Q1:.1f}\n"
            f"Mediana = {Q2:.1f}\n"
            f"Q3 = {Q3:.1f}"
        )
        plt.text(
            i, y_fixed,
            texto,
            ha="center", va="bottom",
            fontsize=10
        )

    plt.ylim(top=49)
    plt.tight_layout()
    plt.savefig('figuras/box3_expectativa_escolaridade.png')
    plt.show()


In [ ]:
rotulos_grau = [
    "Superior completo",
    "Segundo grau completo",
    "Abaixo do segundo grau",
    "Não informado"
]
plot_boxplot_idades_por_instrucao(df_aposentadorias, rotulos_grau)

### Tabelas

In [ ]:
df_aposentadorias['Espécie.1'].value_counts()

In [ ]:
df_aposentadorias.loc[:,
    ['Sexo.', 'Espécie.1']
].value_counts()


In [ ]:
df_aposentadorias['Grau Instrução'].value_counts()

In [ ]:
df_aposentadorias[["grau_simplificado_indice","grau_simplificado"]].value_counts(dropna=False)

In [ ]:
df_aposentadorias["Região"].value_counts(dropna=False)

In [ ]:
contagem_grupos_renda = (
    df_aposentadorias["grupo_renda"]
    .value_counts()
    .sort_index()
)

contagem_grupos_renda

In [ ]:
def tabela_expectativa_por_salario(df):

    df = df.copy()
    df["expectativa_sobrevida"] = pd.to_numeric(df["expectativa_sobrevida"], errors="coerce")

    # Definindo faixas de expectativa de vida
    bins_exp = [0, 5, 10, 15, 20, 25, 30, float("inf")]
    labels_exp = ["[0,5)", "[5,10)", "[10,15)", "[15,20)", "[20,25)", "[25,30)", "30+"]

    df["faixa_expectativa"] = pd.cut(
        df["expectativa_sobrevida"],
        bins=bins_exp,
        labels=labels_exp,
        right=False
    )

    # grupos salariais
    labels_sal = ["[0,2)", "[2,4)", "4+"]

    # tabela de contagem
    contagem = (
        df
        .groupby(["faixa_expectativa", "grupo_renda"])
        .size()
        .unstack(fill_value=0)
        .reindex(index=labels_exp, columns=labels_sal, fill_value=0)
    )

    percentuais = contagem.div(contagem.sum(axis=0), axis=1) * 100
    percentuais = percentuais.round(1).astype(str) + "%"

    tabela_final = contagem.astype(str) + " (" + percentuais + ")"

    return tabela_final


tabela_expectativa_por_salario(df_aposentadorias)

In [ ]:
def resumo_salario_por_instrucao(
    df,
    instrucao_col="grau_simplificado",
    salario_col="grupo_renda",
    instr_labels=None,
    labels_sal=None,
    dropna_instrucao=True,
    dropna_salario=True
):

    df = df.copy()

    if dropna_instrucao:
        df = df[df[instrucao_col].notna()]
    if dropna_salario:
        df = df[df[salario_col].notna()]

    if instr_labels is None:
        instr_labels = sorted(df[instrucao_col].unique())

    if labels_sal is None:
        labels_sal = sorted(df[salario_col].unique())

    counts = (
        df.groupby([instrucao_col, salario_col])
          .size()
          .unstack(fill_value=0)
          .reindex(index=instr_labels, columns=labels_sal, fill_value=0)
    )

    pct = counts.div(counts.sum(axis=1), axis=0) * 100
    pct = pct.fillna(0)

    out_cols = []
    out_parts = []

    for sal in labels_sal:
        out_parts.append(counts[sal].rename(f"count_{sal}"))
        out_parts.append(pct[sal].rename(f"pct_{sal}"))
        out_cols.extend([f"count_{sal}", f"pct_{sal}"])

    result = pd.concat(out_parts, axis=1)
    result["total"] = counts.sum(axis=1)

    result = result[out_cols + ["total"]]

    return result


instr_labels = [
    "Superior completo",
    "Segundo grau completo",
    "Abaixo do segundo grau",
    "Não informado"
]

labels_sal = ["[0,2)", "[2,4)", "4+"]

resumo = resumo_salario_por_instrucao(
    df_aposentadorias,
    instrucao_col="grau_simplificado",
    salario_col="grupo_renda",
    instr_labels=instr_labels,
    labels_sal=labels_sal
)

resumo


### regressão

In [ ]:
def rodar_regressao(df_base, target_col, feature_cols,
                    numeric_features, categorical_features):

    df = df_base[feature_cols + [target_col]].dropna()

    if df.shape[0] < 50:
        print("Registros insuficientes.")
        return None, None, None

    X = df[feature_cols]
    y = df[target_col]

    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore", drop="first"), categorical_features),
            ("num", "passthrough", numeric_features)
        ]
    )

    model = LinearRegression()

    pipeline = Pipeline([
        ("preprocessing", preprocessor),
        ("linreg", model)
    ])

    pipeline.fit(X, y)

    ohe = pipeline.named_steps["preprocessing"].named_transformers_["cat"]
    cat_nomes = ohe.get_feature_names_out(categorical_features)
    feature_nomes = list(cat_nomes) + numeric_features

    coefs = pipeline.named_steps["linreg"].coef_
    intercepto = pipeline.named_steps["linreg"].intercept_

    coef_df = pd.DataFrame({
        "feature": feature_nomes,
        "coeficiente": coefs
    }).sort_values("coeficiente", ascending=False)

    return pipeline, coef_df, intercepto

In [ ]:
pipe_sal, coef_sal, inter_sal = rodar_regressao(
    df_base=df_aposentadorias,
    target_col="Qt SM RMI",
    feature_cols=["Espécie.1", "grau_simplificado_indice", "Sexo.",
                  "Forma Filiação", "Região"],
    numeric_features=["grau_simplificado_indice"],
    categorical_features=["Espécie.1", "Sexo.", "Forma Filiação", "Região"]
)

print(coef_sal)
print("Intercepto:", inter_sal)


In [ ]:
ohe = pipe_sal.named_steps["preprocessing"].named_transformers_["cat"]
for nome, cats in zip(["Espécie.1", "Sexo.", "Forma Filiação", "Região"], ohe.categories_):
    print(nome, "→ baseline =", cats[0], "(dropped)")


In [ ]:
pipe_exp, coef_exp, inter_exp = rodar_regressao(
    df_base=df_aposentadorias,
    target_col="expectativa_sobrevida",
    feature_cols=["Espécie.1", "grau_simplificado_indice", "Sexo.","Qt SM RMI","Região","Forma Filiação"],
    numeric_features=["grau_simplificado_indice","Qt SM RMI"],
    categorical_features=["Espécie.1", "Sexo.", "Região","Forma Filiação"]
)

print(coef_exp)
print("Intercepto:", inter_exp)
